Firstly, I had to import all the libraries I will use. 

In [8]:
!pip install pandas
!pip install statsmodels

In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.sandbox.regression.gmm import IV2SLS


I imported the DESI index used by the European Commission. Unfortunately the data was split into different categories, so I had to first find a list of unique countries used in the data. 

In [10]:
desi = pd.read_csv("C:/Users/rossm/OneDrive/Desktop/python_ws/Dissertation/desi.csv")
print(desi)
desi_country_list = desi.CATEGORY.unique().tolist()
print(desi_country_list)

                      SERIES     CATEGORY CODE      VALUE  NOTE  FLAG
0              Human Capital      Finland   FI  17.847650   NaN   NaN
1              Human Capital      Denmark   DK  14.796675   NaN   NaN
2              Human Capital  Netherlands   NL  15.781750   NaN   NaN
3              Human Capital       Sweden   SE  15.494350   NaN   NaN
4              Human Capital      Ireland   IE  15.660500   NaN   NaN
..                       ...          ...  ...        ...   ...   ...
107  Digital Public Services     Slovakia   SK  12.999450   NaN   NaN
108  Digital Public Services       Poland   PL  13.940675   NaN   NaN
109  Digital Public Services       Greece   EL   9.846400   NaN   NaN
110  Digital Public Services     Bulgaria   BG  12.974125   NaN   NaN
111  Digital Public Services      Romania   RO   5.260825   NaN   NaN

[112 rows x 6 columns]
['Finland', 'Denmark', 'Netherlands', 'Sweden', 'Ireland', 'Malta', 'Spain', 'Luxembourg', 'Estonia', 'Austria', 'Slovenia', 'France', '

Unfortunately the data was split into different categories.The function below sums the values of each seperated category. The function takes a country as an input and filters the dataframe so only one country remains, the VALUE is summed to show all the different categories of one country summed as an aggregated desi. I will use this function later.

In [11]:
def aggregate_desi(country):
    country_desi = desi[desi.CATEGORY == country]
    score = country_desi.VALUE.sum()
    return score

print(aggregate_desi("Finland"))
        

69.59762500000001


Next, I import the data for capital that I will use in my analysis. There was space in the dataframe with empty cells, so I decided to get rid of these and reset the index to make it less confusing for me. I also changed "Czech Repulic" to "Czechia" to make it consistent with my other data. Because my analysis is cross sectional, I only care about the year 2020, so I filter for that year. I also want a list of countries for this dataframe 

In [12]:
capital = pd.read_csv("C:/Users/rossm/OneDrive/Desktop/python_ws/Dissertation/capital_stock.csv")
capital2020 = capital[(capital.Time == 2020) & (capital.Indicator == "Net capital stock, volume, year 2015 = 100")]
capital2020.reset_index(inplace=True, drop = True)
capital2020.loc[4, "Country"] = "Czechia"
print(capital2020)
print(len(capital2020))
capital_country_list = list(capital2020.Country)
print(capital_country_list)

   LOCATION          Country   INDICATOR  \
0       AUS        Australia  AN11NVIXOB   
1       AUT          Austria  AN11NVIXOB   
2       BEL          Belgium  AN11NVIXOB   
3       CAN           Canada  AN11NVIXOB   
4       CZE          Czechia  AN11NVIXOB   
5       FIN          Finland  AN11NVIXOB   
6       FRA           France  AN11NVIXOB   
7       DEU          Germany  AN11NVIXOB   
8       HUN          Hungary  AN11NVIXOB   
9       ITA            Italy  AN11NVIXOB   
10      NLD      Netherlands  AN11NVIXOB   
11      POL           Poland  AN11NVIXOB   
12      SVK  Slovak Republic  AN11NVIXOB   
13      SWE           Sweden  AN11NVIXOB   
14      USA    United States  AN11NVIXOB   
15      EST          Estonia  AN11NVIXOB   
16      SVN         Slovenia  AN11NVIXOB   
17      DNK          Denmark  AN11NVIXOB   
18      GBR   United Kingdom  AN11NVIXOB   
19      GRC           Greece  AN11NVIXOB   
20      LVA           Latvia  AN11NVIXOB   
21      LUX       Luxembourg  AN

My code for importing labour follows the same logic as for importing capital.

In [17]:
labour = pd.read_csv("C:/Users/rossm/OneDrive/Desktop/python_ws/Dissertation/labour.csv")
new_headers = labour.iloc[3]
labour = labour.iloc[4:].reset_index(drop=True)
labour.columns = new_headers
print(labour)
labour_countries_list = list(labour["Country Name"])
print(labour_countries_list)



3                   Country Name Country Code      Indicator Name  \
0                          Aruba          ABW  Labor force, total   
1    Africa Eastern and Southern          AFE  Labor force, total   
2                    Afghanistan          AFG  Labor force, total   
3     Africa Western and Central          AFW  Labor force, total   
4                         Angola          AGO  Labor force, total   
..                           ...          ...                 ...   
261                       Kosovo          XKX  Labor force, total   
262                  Yemen, Rep.          YEM  Labor force, total   
263                 South Africa          ZAF  Labor force, total   
264                       Zambia          ZMB  Labor force, total   
265                     Zimbabwe          ZWE  Labor force, total   

3    Indicator Code  1960.0  1961.0  1962.0  1963.0  1964.0  1965.0  ...  \
0    SL.TLF.TOTL.IN     NaN     NaN     NaN     NaN     NaN     NaN  ...   
1    SL.TLF.TOTL.IN

I noticed a problem with data I started to find on productivity. The data for productivity only included country codes while I had been working with country names.  I could map the country codes of capital to the country names of capital.

In [18]:
codes = capital["LOCATION"]
names = capital["Country"]
code_to_name = dict(zip(codes, names))


The data for the productivity file is imported below. A new column with country names is made from the corresponding country codes. The last two lines of code check that no countries have been missed. Bulgaria, Romania and Croatia have been missed, which means these are not included in the capital dataset. This means these values will be filtered out later, so I will ignore them. 

In [19]:
productivity = pd.read_csv("C:/Users/rossm/OneDrive/Desktop/python_ws/Dissertation/productivity.csv")
print(productivity)
productivity["country"] = productivity["LOCATION"].replace(code_to_name)
print(productivity)
productivity_country_list = productivity["LOCATION"].replace(code_to_name).unique()
print(productivity_country_list)

      LOCATION INDICATOR SUBJECT MEASURE FREQUENCY  TIME      Value Flag Codes
0          AUT  GDPHRWKD     TOT     USD         A  2018  68.309455        NaN
1          AUT  GDPHRWKD     TOT     USD         A  2019  68.309868        NaN
2          AUT  GDPHRWKD     TOT     USD         A  2020  69.970148        NaN
3          AUT  GDPHRWKD     TOT     USD         A  2021  69.634603        NaN
4          BEL  GDPHRWKD     TOT     USD         A  2018  71.681631        NaN
..         ...       ...     ...     ...       ...   ...        ...        ...
117        ROU  GDPHRWKD     TOT     USD         A  2021  37.514761        NaN
118  EU27_2020  GDPHRWKD     TOT     USD         A  2018  53.904831        NaN
119  EU27_2020  GDPHRWKD     TOT     USD         A  2019  54.462041        NaN
120  EU27_2020  GDPHRWKD     TOT     USD         A  2020  55.142460        NaN
121  EU27_2020  GDPHRWKD     TOT     USD         A  2021  55.296144        NaN

[122 rows x 8 columns]
      LOCATION INDICATOR SUB

Data for GDP is imported. Only dataframe is filtered to only included gdp from the year 2020. A unique country list is produced. 

In [20]:
gdp = pd.read_csv("C:/Users/rossm/OneDrive/Desktop/python_ws/Dissertation/gdp.csv")
gdp["country"] = gdp["LOCATION"].replace(code_to_name)
gdp2020 = gdp[gdp["TIME"] == 2020]
print(gdp2020)

gdp_country_list = gdp2020["LOCATION"].replace(code_to_name).unique()
print(gdp_country_list)

    LOCATION INDICATOR SUBJECT  MEASURE FREQUENCY  TIME         Value  \
3        AUS       GDP     TOT  USD_CAP         A  2020  55690.918600   
7        AUT       GDP     TOT  USD_CAP         A  2020  57253.300560   
11       BEL       GDP     TOT  USD_CAP         A  2020  54539.032530   
15       CAN       GDP     TOT  USD_CAP         A  2020  47228.372490   
19       CZE       GDP     TOT  USD_CAP         A  2020  42813.741340   
..       ...       ...     ...      ...       ...   ...           ...   
240      ALB       GDP     TOT  USD_CAP         A  2020  14033.967210   
244      SRB       GDP     TOT  USD_CAP         A  2020  19547.725670   
248      GEO       GDP     TOT  USD_CAP         A  2020  14731.202570   
252      CMR       GDP     TOT  USD_CAP         A  2020   3867.183996   
256      SEN       GDP     TOT  USD_CAP         A  2020   3513.149951   

    Flag Codes         country  
3          NaN       Australia  
7          NaN         Austria  
11           P         B

A list is formed that includes countries where we have data for all the variables we care about. The data is sorted into alphabetical order to match the other dataframes, otherwise the data won't be corresponding.

In [21]:
common_countries = list(set(productivity_country_list) & set(gdp_country_list) & set(labour_countries_list) & set(capital_country_list) & set(desi_country_list))
common_countries.sort()
print(len(common_countries))

19


The code below essential uses the "aggregate desi" function on each country in the data and creates a list of the desi values of countries in alphabetical order.

In [22]:
desi_list = []
for country in common_countries:
    desi_list.append(aggregate_desi(country))
print(desi_list)
print(len(desi_list))

[54.675675, 50.3074, 69.33382499999999, 56.512325000000004, 69.59762500000001, 53.329125000000005, 52.883, 38.931124999999994, 43.759625, 49.25375, 49.710725, 52.714400000000005, 58.851425, 67.369575, 40.547475, 50.756625, 53.370425, 60.772549999999995, 65.223025]
19


The code below filters out any countries that are not common across all datasets, then it sorts the data in the alphabetical order of the countries. "gdp_final" contains only the values of GDP. I would've liked to create as function to do this as there is a lot of repetive code but unfortunately the names of heading differ slightly that will make this difficult. 

In [23]:
print(gdp2020)
gdp_filtered = gdp2020[gdp2020["country"].isin(common_countries)]
gdp_sorted = gdp_filtered.sort_values("country")
print(gdp_sorted)
gdp_final = gdp_filtered["Value"]
print(gdp_final)
print(len(gdp_final))

    LOCATION INDICATOR SUBJECT  MEASURE FREQUENCY  TIME         Value  \
3        AUS       GDP     TOT  USD_CAP         A  2020  55690.918600   
7        AUT       GDP     TOT  USD_CAP         A  2020  57253.300560   
11       BEL       GDP     TOT  USD_CAP         A  2020  54539.032530   
15       CAN       GDP     TOT  USD_CAP         A  2020  47228.372490   
19       CZE       GDP     TOT  USD_CAP         A  2020  42813.741340   
..       ...       ...     ...      ...       ...   ...           ...   
240      ALB       GDP     TOT  USD_CAP         A  2020  14033.967210   
244      SRB       GDP     TOT  USD_CAP         A  2020  19547.725670   
248      GEO       GDP     TOT  USD_CAP         A  2020  14731.202570   
252      CMR       GDP     TOT  USD_CAP         A  2020   3867.183996   
256      SEN       GDP     TOT  USD_CAP         A  2020   3513.149951   

    Flag Codes         country  
3          NaN       Australia  
7          NaN         Austria  
11           P         B

All the other dataframes are filtered and sorted in the same way, taking into account that we want data for 2020 only.

In [24]:
labour_filtered = labour[labour["Country Name"].isin(common_countries)]
print(labour_filtered)
labour_sorted = labour_filtered.sort_values("Country Name")
print(labour_sorted)
labour_final = labour_sorted[2020]
print(labour_final)
print(len(labour_final))

3   Country Name Country Code      Indicator Name  Indicator Code  1960.0  \
14       Austria          AUT  Labor force, total  SL.TLF.TOTL.IN     NaN   
17       Belgium          BEL  Labor force, total  SL.TLF.TOTL.IN     NaN   
55       Germany          DEU  Labor force, total  SL.TLF.TOTL.IN     NaN   
58       Denmark          DNK  Labor force, total  SL.TLF.TOTL.IN     NaN   
70         Spain          ESP  Labor force, total  SL.TLF.TOTL.IN     NaN   
71       Estonia          EST  Labor force, total  SL.TLF.TOTL.IN     NaN   
75       Finland          FIN  Labor force, total  SL.TLF.TOTL.IN     NaN   
77        France          FRA  Labor force, total  SL.TLF.TOTL.IN     NaN   
89        Greece          GRC  Labor force, total  SL.TLF.TOTL.IN     NaN   
101      Hungary          HUN  Labor force, total  SL.TLF.TOTL.IN     NaN   
116        Italy          ITA  Labor force, total  SL.TLF.TOTL.IN     NaN   
143    Lithuania          LTU  Labor force, total  SL.TLF.TOTL.IN     NaN   

In [25]:
print(capital2020)
capital_filtered = capital2020[capital2020["Country"].isin(common_countries)]
capital_sorted = capital_filtered.sort_values("Country")
print(capital_sorted)
capital_final = capital_filtered["Value"]
print(capital_final)
print(len(capital_final))

   LOCATION          Country   INDICATOR  \
0       AUS        Australia  AN11NVIXOB   
1       AUT          Austria  AN11NVIXOB   
2       BEL          Belgium  AN11NVIXOB   
3       CAN           Canada  AN11NVIXOB   
4       CZE          Czechia  AN11NVIXOB   
5       FIN          Finland  AN11NVIXOB   
6       FRA           France  AN11NVIXOB   
7       DEU          Germany  AN11NVIXOB   
8       HUN          Hungary  AN11NVIXOB   
9       ITA            Italy  AN11NVIXOB   
10      NLD      Netherlands  AN11NVIXOB   
11      POL           Poland  AN11NVIXOB   
12      SVK  Slovak Republic  AN11NVIXOB   
13      SWE           Sweden  AN11NVIXOB   
14      USA    United States  AN11NVIXOB   
15      EST          Estonia  AN11NVIXOB   
16      SVN         Slovenia  AN11NVIXOB   
17      DNK          Denmark  AN11NVIXOB   
18      GBR   United Kingdom  AN11NVIXOB   
19      GRC           Greece  AN11NVIXOB   
20      LVA           Latvia  AN11NVIXOB   
21      LUX       Luxembourg  AN

In [26]:
print(productivity)
prod_2020 = productivity[productivity["TIME"] == 2020]
print(prod_2020)
prod_filtered = prod_2020[prod_2020["country"].isin(common_countries)]
print(prod_filtered)
prod_sorted = prod_filtered.sort_values("country")
prod_final = prod_sorted["Value"]
print(list(prod_final))

      LOCATION INDICATOR SUBJECT MEASURE FREQUENCY  TIME      Value  \
0          AUT  GDPHRWKD     TOT     USD         A  2018  68.309455   
1          AUT  GDPHRWKD     TOT     USD         A  2019  68.309868   
2          AUT  GDPHRWKD     TOT     USD         A  2020  69.970148   
3          AUT  GDPHRWKD     TOT     USD         A  2021  69.634603   
4          BEL  GDPHRWKD     TOT     USD         A  2018  71.681631   
..         ...       ...     ...     ...       ...   ...        ...   
117        ROU  GDPHRWKD     TOT     USD         A  2021  37.514761   
118  EU27_2020  GDPHRWKD     TOT     USD         A  2018  53.904831   
119  EU27_2020  GDPHRWKD     TOT     USD         A  2019  54.462041   
120  EU27_2020  GDPHRWKD     TOT     USD         A  2020  55.142460   
121  EU27_2020  GDPHRWKD     TOT     USD         A  2021  55.296144   

    Flag Codes    country  
0          NaN    Austria  
1          NaN    Austria  
2          NaN    Austria  
3          NaN    Austria  
4      

All the data is then organised into a neat dataframe, but with only the data I actually want to use. 

In [27]:
# create final df
final_df = pd.DataFrame({
    "country": common_countries,
    "labour": labour_final,
    "DESI": desi_list,
    "GDP": list(gdp_final),
    "capital": list(capital_final),
    "productivity": list(prod_final)
})

print(final_df)

         country      labour       DESI           GDP     capital  \
14       Austria   4638300.0  54.675675   57253.30056  107.527495   
17       Belgium   5168148.0  50.307400   54539.03253  108.356983   
58       Denmark   3028252.0  69.333825   60840.94016  107.373047   
71       Estonia    706678.0  56.512325   52289.30316  105.814214   
75       Finland   2752889.0  69.597625   47823.91958  104.215454   
77        France  30379166.0  53.329125   56476.93431  108.788504   
55       Germany  44182576.0  52.883000   28413.78734   99.631874   
89        Greece   4682727.0  38.931125   34156.84004  107.196714   
101      Hungary   4740236.0  43.759625   43129.89668  114.213050   
116        Italy  25126336.0  49.253750  119871.42940  111.509484   
145       Latvia    988585.0  49.710725   59815.50154  118.801299   
143    Lithuania   1486169.0  52.714400   34893.45977   99.372142   
144   Luxembourg    322041.0  58.851425   34952.24306  111.295010   
176  Netherlands   9502135.0  67.3

All the values are logged so I have the form of my regression equation. This makes my analysis reflect the theory I've written in my report. 

In [28]:
#log the values
final_df["log_Y"] = np.log(final_df["GDP"])
final_df["log_K"] = np.log(final_df["capital"])
final_df["log_desi"] = np.log(final_df["DESI"])
final_df["log_A"] = np.log(final_df["productivity"])
final_df["log_L"] = np.log(final_df["labour"])


The regression is formed and executed in the lines of code below. 

In [29]:
#REGRESSION
X = final_df[["log_desi", "log_K", "log_A", "log_L"]]
y = final_df["log_Y"]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
print(results.params)

const       3.927541
log_desi   -0.149145
log_K       1.385131
log_A       0.115901
log_L       0.031237
dtype: float64


Here are the results below. None of the coefficient estimates are statistically significant, so I can't prove there's an association between the dependent and independent variables.

In [31]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  log_Y   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                 -0.157
Method:                 Least Squares   F-statistic:                    0.3891
Date:                Sat, 16 Sep 2023   Prob (F-statistic):              0.813
Time:                        20:14:50   Log-Likelihood:                -4.1822
No. Observations:                  19   AIC:                             18.36
Df Residuals:                      14   BIC:                             23.09
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.9275      7.053      0.557      0.5

C:\Users\rossm\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


A Breusch-Pagan test will test the heteroskedasticity of the model. Our model is found to be heteroskedastic, which means the variance of the error term changes as independent variables change. This means that coefficient estimates are inefficient. Heteroskedasticity inflates the variance variance of estimators making less reliable.

In [32]:
# Breusch-Pagan test
names = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test_results = sms.het_breuschpagan(results.resid, results.model.exog)
lzip(names, test_results)

[('Lagrange multiplier statistic', 5.104279386898763),
 ('p-value', 0.2767641606666139),
 ('f-value', 1.2856460166090355),
 ('f p-value', 0.3223768180538159)]